### Сравнение собственной реализации SVM и готовой реализации из библиотеки scikit-learn

Сравним две реализации метода опорных векторов. Сравнения будут проводится на датасетах из 0-ой лабораторной работы. В качестве метрики качества возьмем точность - отношение правильных ответов к количеству всех элементов в тестовой выорки, также рассмотрим время обучения. Для этого определим функцию test_classifier
    

In [1]:
from time import time
from sklearn.metrics import accuracy_score

def test_classifier(cls, X_train, y_train, X_test, y_test):
    start = time()
    cls.fit(X_train, y_train)
    end = time()
    y_pred = cls.predict(X_test)
    print("Time:", end - start)
    print("Accuracy:", accuracy_score(y_true=y_test, y_pred=y_pred))

Перед тестированием алгоритмов на первом датасете необходимо выполнить его предобработку. Так как метод опорных векторов, как и другие линейные классификаторы чувствительны к масштабу признаков, необходимо провести нормализацию - вычесть среднее и поделить на дисперсию. Так как датасет довольно большой, возьмем только часть записей: 

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv('diamonds.csv', index_col="index")[:5000]
# Выделение целевой переменной
y = data['price'].to_numpy()
del data['price']
# Выделение матрицы объект-признак и масштабирование
scaler = StandardScaler()
X = scaler.fit_transform(data.values)
# Разбиение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=126)

Результат тестирования первой реализации:

In [5]:
from SupportVectorMachine import SupportVectorClassifier

model = SupportVectorClassifier(C=1.0, max_iterations=10000, tolerance=0.001)
test_classifier(model, X_train, y_train, X_test, y_test)

Time: 12.251717329025269
Accuracy: 0.785


Результат тестирования второй реализации:

In [7]:
from sklearn.svm import SVC

model = SVC(C=1.0, kernel='linear', tol=0.001)
test_classifier(model, X_train, y_train, X_test, y_test)

Time: 0.399141788482666
Accuracy: 0.81


Как видно, моя реализация сильно уступает во времени обучения и немного в точности. Длительное время выполнения на довольно малой части датасета связано скорее всего с упрощениями в реализации алгоритма оптимизации. Не использовались некоторые эвристики выбора элементов на каждой итерации, а также не эффективный пересчет весов (В данном случае, можно было воспользоваться линейностью ядра, и обновлять веса, а не пересчитывать их заного)

Рассмотрим, как поведет себя SVM в случае классификации документов. Данная задача примечательно тем, что размерность пространства объектов гораздо больше их количества. Подготовим данные для тестирования

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

targets = ['comp.graphics', 'rec.autos']
documents = fetch_20newsgroups(data_home='./', subset='all', categories=targets)

documents.target[documents.target == 0] = -1
y = documents.target[:100]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents.data[:100]).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=126)

In [14]:
model = SupportVectorClassifier(C=1.0, max_iterations=10000, tolerance=0.01)
test_classifier(model, X_train, y_train, X_test, y_test)

Time: 1403.578275680542
Accuracy: 0.9363867684478372


In [13]:
model = SVC(C=1.0, kernel='linear', tol=0.001)
test_classifier(model, X_train, y_train, X_test, y_test)

Time: 33.042667627334595
Accuracy: 0.9720101781170484


Не смотря на малое количество документов, обучение проходит действительно долго из-за большой размерности пространства. И разница во времени работы становится уже критичным показателем

### Полиномиальная регрессия



In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv('diamonds.csv', index_col="index")
del data['price']

X = data.values
# Столбец цен для задачи регрессии
y = pd.read_csv('prices.csv', index_col='index')['price'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=126)

from PolynomialRegression import PolynomialRegression

reg = PolynomialRegression(3)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))
print(y_pred[:10].astype(int))
print(y_test[:10])

MSE: 552879.9211515883
[ 2496   906  6529   796  1444  3145  3326   936   777 12835]
[ 2678   717  6947   726  1715  3492  3380   847   679 11640]


При использовании всей выборки была получена огромная ошибка при тестировании, однако полученные значения довольно близки к реальным данным, хотя на отдельных примерах разница слишком высока)